In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import os
import pix2pix
import cv2
import matplotlib.pyplot as plt
from pix2pix import models
import tqdm
from pix2pix import config

In [ ]:
gt_path = 'videos/input_video_gt.mp4'
noisy_path = 'videos/input_video_noisy.mp4'
pred_save_path = 'videos/predicted_video_v1.mp4'

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'

In [ ]:
state_dict = torch.load('../models/pix2pix_32_11112021_051302/checkpoints/gen_epoch_4_pix2pix_32_11112021_051302.pt')
model = models.unet(norm_layer=torch.nn.InstanceNorm2d, **config.model.GEN_ARGS)
model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)
model.eval();

In [ ]:
%%time 

noisy_cap = cv2.VideoCapture(noisy_path)
fps = noisy_cap.get(cv2.CAP_PROP_FPS)
width = int(noisy_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(noisy_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
frame_list = []

while noisy_cap.isOpened():
    ret, frame = noisy_cap.read()
    
    if ret == False:
        break
        
    frame = cv2.resize(frame, (256, 256))
    frame_list.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
noisy_cap.release()

fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use lower case
out = cv2.VideoWriter(pred_save_path, fourcc, fps, (width, height))

for image in tqdm.tqdm(frame_list):
    inp = torch.Tensor(image).to(device)
    inp = (inp / (255 / 2)) - 1
    inp = inp.permute((2, 0, 1))
    inp = inp.unsqueeze(0)
    
    pred = model(inp).detach()
    pred = (pred + 1) / 2 
    pred = pred.squeeze()
    pred = pred.permute(1, 2, 0)
    pred = pred.to('cpu')
    pred = (pred.numpy() * 255).astype('uint8')
    pred = cv2.resize(pred, (width, height))
    
#     cv2.imshow('asdads', pred)
#     cv2.waitKey(2)
    
    out.write(cv2.cvtColor(pred, cv2.COLOR_RGB2BGR))
    
out.release()